In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import sys
import statsmodels.api as sm


In [4]:
df=pd.read_csv('hpi_master.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127329 entries, 0 to 127328
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   hpi_type    127329 non-null  object 
 1   hpi_flavor  127329 non-null  object 
 2   frequency   127329 non-null  object 
 3   level       127329 non-null  object 
 4   place_name  127329 non-null  object 
 5   place_id    127329 non-null  object 
 6   yr          127329 non-null  int64  
 7   period      127329 non-null  int64  
 8   index_nsa   127328 non-null  float64
 9   index_sa    42525 non-null   float64
dtypes: float64(2), int64(2), object(6)
memory usage: 9.7+ MB


,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00,100.00
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.89,100.94
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.29,100.90
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.68,100.96
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.30,101.31


In [5]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].apply(lambda x: x if pd.notna(x) else np.random.choice(df[col].dropna().values))

df.fillna(df.mean(), inplace=True)  # Fill numeric missing values with mean

df = pd.get_dummies(df, drop_first=True)

# Heatmap: Feature Correlations
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()

# Histogram: House Price Distribution
plt.figure(figsize=(10, 6))
sns.histplot(df["price"].dropna(), kde=True, bins=30)  # Drop NaNs for safety
plt.xlabel("House Price")
plt.ylabel("Frequency")
plt.title("Distribution of House Prices")
plt.show()

KeyError: 'price'